In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataLoad import PulsarData
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import shap
from skorch import NeuralNetClassifier

In [2]:
# For pretty plotting
plt.style.use('seaborn-paper')
plt.rcParams["font.family"] = "serif"

Creating the neural network:

In [3]:
class NeuralN(nn.Module):
    def __init__(self,inputsize,hiddensize):
        super(NeuralN, self).__init__()
        self.inputsize=inputsize
        self.hiddensize=hiddensize
        # an affine operation: y = Wx + b, this is basically a weight tensor!
        self.fcinput = nn.Linear(in_features=self.inputsize, out_features=self.hiddensize)
        self.fcoutput = nn.Linear(in_features=self.hiddensize, out_features=2)
    
    def forward(self,x: torch.Tensor):
        x = x.to(dtype=torch.float)
        x = self.fcinput(x)
        x = F.relu(x)
        x = self.fcoutput(x)
        return x

Loading the data:

In [4]:
raw_features = PulsarData('HTRU_2').features
raw_targets = PulsarData('HTRU_2').targets

Defining the epochs for the neural network to train:

In [5]:
epochs = 10

Splitting data into test and train data:

In [6]:
train_features_data, test_features_data, train_targets_data, test_targets_data  =  train_test_split( raw_features, 
                                                        raw_targets, test_size=0.25, random_state=42)

Writing a cross validation function that is compatible with torch: 

In [7]:
def NN_CrossValidation(hiddensize, learning_rate, data, targets):
   cv = 3
   net = NeuralN(data.shape[1], hiddensize)
   dlist = np.array_split(data.to_numpy(), cv)
   tlist = np.array_split(targets.to_numpy(), cv)
   cval = list()
   for d, dat in tqdm(enumerate(dlist)):
      cross_dlist =  dlist[:d] + dlist[d+1 :]
      cross_tlist =  tlist[:d] + tlist[d+1 :]
      cross_dat = torch.from_numpy(np.concatenate(cross_dlist)).float()
      cross_tar = torch.from_numpy(np.concatenate(cross_tlist)).long()
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
      net.train()
      for e in range(epochs):
         epoch_losses = list()
         for n in range(cross_dat.shape[0]):
            net.zero_grad()
            optimizer.zero_grad() 
            prediction = net(cross_dat[n]).unsqueeze(0)
            target = cross_tar[n].unsqueeze(0)
            # Calculating the loss function
            loss = criterion(prediction,target)
            epoch_losses.append(float(loss))
            # Calculating the gradient
            loss.backward()
            optimizer.step()
      net.eval()
      cross_pred = torch.argmax(net(torch.from_numpy(dat).float()),dim=1)
      acc_cross = torch.mean((cross_pred == torch.from_numpy(tlist[d]).long()).float())
      cval.append(acc_cross)

   return np.mean(np.array(cval))

In [8]:
def optimize_NN(data, targets, pars, n_iter=5):
    """Apply Bayesian Optimization to Neural Network parameters."""
    
    def crossval_wrapper(hiddensize, learning_rate):
        """Wrapper of Neural Network cross validation. 
           Notice how we ensure params are casted to integer before we pass them along.
        """
        return NN_CrossValidation(hiddensize=int(hiddensize), 
                                            learning_rate=learning_rate, 
                                            data=data, 
                                            targets=targets)

    boptimizer = BayesianOptimization(f=crossval_wrapper, 
                                     pbounds=pars, 
                                     random_state=42, 
                                     verbose=2)
    boptimizer.maximize(init_points=4, n_iter=n_iter)

    return boptimizer


In [9]:
parameters_BayesianOptimization = {"hiddensize": (10, 500), 
                                   "learning_rate": (0.00001, 0.5),
                                  }

BayesianOptimization = optimize_NN(raw_features, 
                                             raw_targets, 
                                             parameters_BayesianOptimization, 
                                             n_iter=5)
print(BayesianOptimization.max)

0it [00:00, ?it/s]|   iter    |  target   | hidden... | learni... |
-------------------------------------------------
3it [07:21, 147.14s/it]
0it [00:00, ?it/s]|  1        |  0.9084   |  193.5    |  0.4754   |
3it [07:19, 146.40s/it]
0it [00:00, ?it/s]|  2        |  0.9084   |  368.7    |  0.2993   |
3it [04:39, 93.27s/it]
0it [00:00, ?it/s]|  3        |  0.9306   |  86.45    |  0.07801  |
3it [04:39, 93.15s/it]
|  4        |  0.9084   |  38.46    |  0.4331   |
3it [06:06, 122.28s/it]
0it [00:00, ?it/s]|  5        |  0.9665   |  500.0    |  1e-05    |
3it [06:54, 138.22s/it]
|  6        |  0.9084   |  496.7    |  0.1936   |
3it [06:14, 124.69s/it]
|  7        |  0.965    |  284.5    |  1e-05    |
3it [06:21, 127.14s/it]
|  8        |  0.9084   |  141.3    |  0.4997   |
3it [06:45, 135.29s/it]|  9        |  0.9084   |  327.5    |  0.5      |
{'target': 0.9664767384529114, 'params': {'hiddensize': 500.0, 'learning_rate': 1e-05}}



Creating a neural network with the optimal hiddensize:

In [10]:
#print(BayesianOptimization.max['params']['learning_rate'])
net = NeuralNetClassifier(NeuralN(inputsize=raw_features.shape[1], hiddensize=int(BayesianOptimization.max['params']['hiddensize'])), max_epochs=epochs, lr=BayesianOptimization.max['params']['learning_rate'],iterator_train__shuffle=True)

In [11]:
#features_data = PulsarData('HTRU_2').features
#targets_data = PulsarData('HTRU_2').targets
scores = cross_val_score(net, raw_features.to_numpy(), raw_targets.to_numpy(), cv=3, scoring='f1') 
print(f"{scores.mean():.4f} accuracy with a standard deviation of {scores.std():.4f}")
print(abe)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.0917           nan  0.4672
      2           nan       0.0917           nan  0.3189
      3           nan       0.0917           nan  0.3929
      4           nan       0.0917           nan  0.3266
      5           nan       0.0917           nan  0.3534
      6           nan       0.0917           nan  0.3414
      7           nan       0.0917           nan  0.3289
      8           nan       0.0917           nan  0.3110
      9           nan       0.0917           nan  0.4295
     10           nan       0.0917           nan  0.4318
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.0917           nan  0.4354
      2           nan       0.0917           nan  0.3482
      3           nan       0.0917           nan  0.3083
      4           nan       0.0

NameError: name 'abe' is not defined

Converting data into torch tensors:

In [21]:
train_features_data, test_features_data = torch.from_numpy(train_features_data.to_numpy()).float(), torch.from_numpy(test_features_data.to_numpy()).float()
train_targets_data, test_targets_data = torch.from_numpy(train_targets_data.to_numpy()).long(), torch.from_numpy(test_targets_data.to_numpy()).long()

AttributeError: 'Tensor' object has no attribute 'to_numpy'

Setting the optimal learning rate and training the network:

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=BayesianOptimization.max['params']['learning_rate'])
net.train()

NeuralN(
  (fcinput): Linear(in_features=8, out_features=49, bias=True)
  (fcoutput): Linear(in_features=49, out_features=2, bias=True)
)

In [13]:
epochs = 10
for e in range(epochs):
    epoch_losses = list()
    for n in range(train_features_data.shape[0]):
        net.zero_grad()
        optimizer.zero_grad() 
        prediction = net(train_features_data[n]).unsqueeze(0)
        target = train_targets_data[n].unsqueeze(0)
        # Calculating the loss function
        loss = criterion(prediction,target)
        epoch_losses.append(float(loss))
        # Calculating the gradient
        loss.backward()
        optimizer.step()
    print(e, np.mean(epoch_losses))

net.eval()

0 0.11904058890842346
1 0.1070697266854844
2 0.10629891285761564
3 0.10584857639593982
4 0.1047456630957333
5 0.10412013706289666
6 0.10293075028365872
7 0.10230069415726191
8 0.10131413117006917
9 0.10080707330719037


NeuralN(
  (fcinput): Linear(in_features=8, out_features=49, bias=True)
  (fcoutput): Linear(in_features=49, out_features=2, bias=True)
)

Final result for train data and test data:

In [14]:
train_prediction = torch.argmax(net(train_features_data),dim=1)
acc_train = torch.mean((train_prediction == train_targets_data).float())
test_prediction = torch.argmax(net(test_features_data),dim=1)
acc_test = torch.mean((test_prediction == test_targets_data).float())

print(acc_train, acc_test)

tensor(0.9770) tensor(0.9774)


In [15]:
shap_values = shap.DeepExplainer(net, train_features_data   ).shap_values(train_features_data)
shap.summary_plot(shap_values, train_features_data, plot_type="bar")

KeyboardInterrupt: 